# Setup to train full model with findinng proper weights to train each head (Classification + Segmentation)
## Breast-Ultrasound-Segmentation

## About Dataset
Breast cancer is one of the most common causes of death among women worldwide. Early detection helps in reducing the number of early deaths. The data reviews the medical images of breast cancer using ultrasound scan. Breast Ultrasound Dataset is categorized into three classes: normal, benign, and malignant images. Breast ultrasound images can produce great results in classification, detection, and segmentation of breast cancer when combined with machine learning.

### Data
The data collected at baseline include breast ultrasound images among women in ages between 25 and 75 years old. This data was collected in 2018. The number of patients is 600 female patients. The dataset consists of 780 images with an average image size of 500*500 pixels. The images are in PNG format. The ground truth images are presented with original images. The images are categorized into three classes, which are normal, benign, and malignant.

If you use this dataset, please cite:
Al-Dhabyani W, Gomaa M, Khaled H, Fahmy A. Dataset of breast ultrasound images. Data in Brief. 2020 Feb;28:104863. DOI: 10.1016/j.dib.2019.104863.

## Imports

In [1]:
import os

import pyrootutils

root = pyrootutils.setup_root(
    search_from=os.path.dirname(os.getcwd()),
    indicator=[".git", "pyproject.toml"],
    pythonpath=True,
    dotenv=True,
)

if os.getenv("DATA_ROOT") is None:
    os.environ["DATA_ROOT"] = f"{root}/data"

In [2]:
import torch
import torch.nn as nn

# Setup device-agnostic code
if torch.cuda.is_available():
    DEVICE = "cuda"  # NVIDIA GPU
    print("GPU Found!!")
else:
    raise Exception("No GPU Found!!")

GPU Found!!


In [3]:
import logging

import hydra
from hydra import compose, initialize

log = logging.getLogger(__name__)

In [4]:
# # auto reload dotenv
%load_ext dotenv
%dotenv

# auto reload libs
%load_ext autoreload
%autoreload 2

## Paths setup

In [5]:
from omegaconf import DictConfig, OmegaConf

# Register a resolver for torch dtypes
OmegaConf.register_new_resolver("torch_dtype", lambda name: getattr(torch, name))

In [6]:
with initialize(config_path="../configs", job_name="training_setup", version_base=None):
    cfg: DictConfig = compose(config_name="train.yaml")
    # print(OmegaConf.to_yaml(cfg))
    print(cfg)

{'task_name': 'overfitting-tests', 'tags': ['dev'], 'train': True, 'test': False, 'ckpt_path': None, 'seed': 42, 'data': {'dataset': {'_target_': 'src.datamodules.components.breast_cancer_dataset.BreastCancerDataset', 'data_dir': '${data.dataset_dir}', 'dataset_url': '${data.url}'}, 'train_shared_transforms': [{'_target_': 'torchvision.transforms.v2.Resize', 'size': [224, 224], 'antialias': True}, {'_target_': 'torchvision.transforms.v2.RandomHorizontalFlip'}, {'_target_': 'torchvision.transforms.v2.RandomVerticalFlip'}, {'_target_': 'torchvision.transforms.v2.ElasticTransform', 'alpha': 50.0, 'sigma': 5.0}], 'train_image_trasforms': [{'_target_': 'torchvision.transforms.v2.ToDtype', 'dtype': "${torch_dtype:'float32'}", 'scale': True}, {'_target_': 'torchvision.transforms.v2.Normalize', 'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}], 'train_masks_transforms': [{'_target_': 'torchvision.transforms.v2.ToDtype', 'dtype': "${torch_dtype:'float32'}", 'scale': False}], 'val_sh

In [7]:
os.chdir(root)

## Loading Dataset

In [8]:
data_module = hydra.utils.instantiate(cfg.datamodule)

class_weights = data_module.class_weights
class_names = data_module.classes
num_classes = len(class_names)
class_names, num_classes, class_weights

(['normal', 'malignant', 'benign'], 3, tensor([1.9774, 1.2494, 0.5903]))

In [9]:
train_dl, val_dl = data_module.get_sampled_dataloader()

Sampled dataset size: 315


In [10]:
images, targets = next(iter(train_dl))
print(images.shape, targets["masks"].shape, targets["labels"].shape)

print(f"images:{images.dtype}, {images[0].min()}, {images[0].max()}")
print(f'masks {targets["masks"].dtype}, {targets["masks"][0].min()}, {targets["masks"][0].max()}')
print(f'labels {targets["labels"].dtype}, {targets["labels"].min()}, {targets["labels"].max()}')

torch.Size([32, 3, 224, 224]) torch.Size([32, 1, 224, 224]) torch.Size([32])
images:torch.float32, -2.1179039478302, 2.239128828048706
masks torch.uint8, 0, 0
labels torch.int64, 0, 2


In [11]:
_images, _targets = next(iter(val_dl))

print(_images.shape, _targets["masks"].shape, _targets["labels"].shape)

print(f"images:{_images[0].dtype}, {_images[0].min()}, {_images[0].max()}")
print(f'masks {_targets["masks"].dtype}, {_targets["masks"].min()}, {_targets["masks"].max()}')
print(f'labels {_targets["labels"].dtype}, {_targets["labels"].min()}, {_targets["labels"].max()}')

torch.Size([32, 3, 224, 224]) torch.Size([32, 1, 224, 224]) torch.Size([32])
images:torch.float32, -2.1179039478302, 2.640000104904175
masks torch.uint8, 0, 1
labels torch.int64, 0, 2


## Loading and training the FCN8 model 

In [12]:
segmentation_criterion = hydra.utils.instantiate(cfg.losses.segmentation_criterion)
classification_criterion = hydra.utils.instantiate(
    cfg.losses.classification_criterion, weight=class_weights
)
classification_criterion.weight

tensor([1.9774, 1.2494, 0.5903])

In [13]:
import mlflow
import mlflow.pytorch

from src.utils.gpu_utils import DeviceDataLoader, get_default_device, to_device

torch.cuda.empty_cache()
device = get_default_device()

gpu_weights = to_device(class_weights, device)

In [14]:
cfg.models

{'optimizer': {'_target_': 'torch.optim.Adam', 'lr': 0.003}, 'scheduler': {'func': None}, 'model': {'_target_': 'src.models.vggnet_fcn_segmentation_model.VGGNetFCNSegmentationModel', 'segmentation_criterion': None, 'classification_criterion': None, 'seg_num_classes': 1, 'cls_num_classes': 3, 'seg_weight': 0.95, 'cls_weight': 0.05, 'vggnet_type': 'vgg16', 'fcn_type': 'fcn8'}}

In [15]:
model = hydra.utils.instantiate(
    cfg.models.model,
    segmentation_criterion=segmentation_criterion,
    classification_criterion=classification_criterion,
)

In [16]:
model = torch.compile(model)
model

OptimizedModule(
  (_orig_mod): VGGNetFCNSegmentationModel(
    (segmentation_criterion): SoftDiceLoss()
    (classification_criterion): CrossEntropyLoss()
    (cls_auroc): MulticlassAUROC()
    (encoder): VGGNetEncoder(
      (vgg): VGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): ReLU(inplace=True)
          (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (6): ReLU(inplace=True)
          (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): ReLU(inplace=True)
          (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   

In [ ]:
from mlflow.models import infer_signature

task_name = cfg.task_name
mlflow.set_experiment(f"overfitting-{task_name}")
run = mlflow.start_run()
model.eval()  # This ensures layers like Dropout and BatchNorm behave correctly for inference and saves computation.
with torch.no_grad():
    images, labels = next(iter(train_dl))
    # print(images.shape, labels)
    out = model(images)
    # l = labels['labels'][0]
    # print(l , torch.argmax(l))
    print(out["labels"][0], out["labels"].shape)
    print(out["masks"][0])
    print(out["masks"].shape)
    signature = infer_signature(
        model_input={"image_input": images.numpy()},
        model_output={"output": {"masks": out["masks"].numpy(), "labels": out["labels"].numpy()}},
    )
signature

tensor([-0.0485, -0.0661, -0.0109]) torch.Size([32, 3])
tensor([[[-0.1150, -0.1726, -0.1475,  ..., -0.1662, -0.1486, -0.0939],
         [-0.0909, -0.1174, -0.1678,  ..., -0.1083, -0.1540, -0.0844],
         [-0.1602, -0.1890, -0.1252,  ..., -0.2339, -0.1050, -0.1089],
         ...,
         [-0.0912, -0.2368, -0.2086,  ..., -0.1636, -0.0718, -0.1317],
         [-0.0959, -0.2723, -0.2127,  ..., -0.1324, -0.1128, -0.1306],
         [-0.1376, -0.1892, -0.2293,  ..., -0.1414, -0.1422, -0.1360]]])
torch.Size([32, 1, 224, 224])


inputs: 
  ['image_input': Tensor('float32', (-1, 3, 224, 224))]
outputs: 
  ['output': {labels: Array(Array(float)) (required), masks: Array(Array(Array(Array(float)))) (required)} (required)]
params: 
  None

## GPU Training Setup

## Moving data and model into memory

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)
# train_dl.device

## Overfiting the model

In [ ]:
optimizer = hydra.utils.instantiate(cfg.models.optimizer, params=model.parameters(), lr=1e-4)

In [25]:
cfg.paths

{'root_dir': '${oc.env:PROJECT_ROOT}', 'root_data_dir': '${oc.env:DATA_ROOT}', 'log_dir': '${paths.root_dir}/logs/', 'output_dir': '${hydra:runtime.output_dir}', 'work_dir': '${hydra:runtime.cwd}'}

In [ ]:
cfg.paths.output_dir

In [ ]:
from torchinfo import summary

from src.utils.train_utils import fit

EPOCHS = cfg.trainer.max_epochs
mlflow.log_params({"epochs": EPOCHS})
mlflow.log_params({"batch_size": cfg.datamodule.batch_size})
mlflow.log_params({"optimizer": cfg.models.optimizer.values()})
# Log model summary.
with open("model_summary.txt", "w") as f:
    f.write(str(summary(model)))
mlflow.log_artifact("model_summary.txt")
history = fit(
    model=model,
    train_dataloader=train_dl,
    validation_dataloader=val_dl,
    epochs=EPOCHS,
    optimizer=optimizer,
    device_type=device.type,
    dtype=torch.float16,
    reduce_lr_on_plateau=torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, factor=0.1, patience=5
    ),
)
# saving the trained model
mlflow.pytorch.log_model(model, "model", signature=signature)
mlflow.log_metrics(history[0])

In [19]:
mlflow.end_run()

In [ ]:
import matplotlib.pyplot as plt

seg_losses = [x["seg_loss"] for x in history]
seg_dice = [x["seg_dice"] for x in history]

plt.plot(seg_losses, "-bx")
plt.plot(seg_dice, "-rx")

plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid()
plt.legend(["seg_loss", "seg_dice"])
plt.title("Loss vs. NO. of epochs")